In [1]:
# Mapas nacionales y subnacionales.

import os

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import colors

import geopandas as gpd

In [24]:
import xarray as xr
import rioxarray as rio
f = "VIENTO_MUNDO_TR100_INT1.grd"
ds = rio.open_rasterio(f)
ds["x"].values[1] - ds["x"].values[2]
ds_f = ds * 1.1

In [25]:
ds_f.where(ds_f>=252).count()

<xarray.DataArray ()> Size: 8B
array(27194)
Coordinates:
    spatial_ref  int64 8B 0

In [26]:
ds.where(ds>=252).count()

<xarray.DataArray ()> Size: 8B
array(17333)
Coordinates:
    spatial_ref  int64 8B 0

In [14]:
# Datos

# Carpetas
path_catalog = "../../Bases_de_datos/Data_catalog.csv"
data_d = "../results/"
wb_path = "../../Bases_de_datos/World_Bank_CCKP/"
iso = "../../Bases_de_datos/Country_ISO_code.csv"

# Códigos nacionales y subnacionales
ix  = "alpha-3"
ix1 = "iso_3166_2" 

# Relación de códigos Banco Mundial y Natural Earth (ISO 3166-2)
code_path = "../../Bases_de_datos/Subnational_Code_WB_ISO.csv"

# Catálogo de datos
df_c = pd.read_csv(path_catalog)


df_iso = pd.read_csv(iso).set_index(ix)

df_c.head()

,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
0,Exposure to sea level rise,pone.0118571.s004,Neumann et al.,All,National,Periods,NaN,https://doi.org/10.1371/journal.pone.0118571,pone.0118571.s004.csv,Bases_de_datos/Sea_level_rise/
1,Air Emissions Accounts,DF_AEA,OECD,All,National,"OECD: Quarterly, Non-OECD: Yearly",NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AEA@DF_AEA,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
2,Air emissions - Air pollutants Inventories,DF_AIR_EMISSIONS,OECD,All,National,Yearly,Air Emissions Correspondance Table.xlsx,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_EMISSIONS@DF_AIR_EMISSION...",Bases_de_datos/OECD_Air_and_Climate/
3,Air emissions - Greenhouse gas emissions Inven...,DF_AIR_GHG,OECD,All,National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_GHG@DF_AIR_GHG,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
4,Air transport CO2 emissions (experimental),DF_AIR_TRANSPORT,OECD,All,National,Monthly,NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AIR_TRANSPORT@DF_AIR_TRA...",Bases_de_datos/OECD_Air_and_Climate/


In [23]:
id = "pone.0118571.s004"
col = [
    "Major area, region, country or area",
    "Baseline - total population 2000 [thousand]",
    "Baseline - LECZ total population 2000",
    "Scenario D - Total population 2060 [thousand]",
    "Scenario D - LECZ total population 2060",
    ]

# Cargamos el archivo
df = pd.read_csv( "../../"
    + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0],
    index_col = ix, )
df = df.loc[ df.index != "-99", col ]

v = "% Population exposed to sea level rise, change 2000-2060"

df["% Population 2000"] = 100 * df[col[2]] / ( df[col[1]] * 1000 )
df["% Population 2060"] = 100 * df[col[4]] / ( df[col[3]] * 1000 )
df_iso[v] = df["% Population 2060"] - df["% Population 2000"]

df_iso.sort_values(v, ascending = False).head(10)

,name,alpha-2,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,OECD,EU27,BRICS+,BRICS,"% Population exposed to sea level rise, hange 2000-2060"
alpha-3,,,,,,,,,,,,,,,
BEN,Benin,BJ,204,ISO 3166-2:BJ,Africa,Sub-Saharan Africa,Western Africa,2.0,202.0,11.0,NaN,NaN,NaN,NaN,23.751000
TTO,Trinidad and Tobago,TT,780,ISO 3166-2:TT,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0,NaN,NaN,NaN,NaN,22.591340
QAT,Qatar,QA,634,ISO 3166-2:QA,Asia,Western Asia,NaN,142.0,145.0,NaN,NaN,NaN,NaN,NaN,20.770959
THA,Thailand,TH,764,ISO 3166-2:TH,Asia,South-eastern Asia,NaN,142.0,35.0,NaN,NaN,NaN,NaN,NaN,17.793734
SEN,Senegal,SN,686,ISO 3166-2:SN,Africa,Sub-Saharan Africa,Western Africa,2.0,202.0,11.0,NaN,NaN,NaN,NaN,13.893504
ABW,Aruba,AW,533,ISO 3166-2:AW,Americas,Latin America and the Caribbean,Caribbean,19.0,419.0,29.0,NaN,NaN,NaN,NaN,12.879192
SOM,Somalia,SO,706,ISO 3166-2:SO,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0,NaN,NaN,NaN,NaN,12.289061
WLF,Wallis and Futuna,WF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN,NaN,NaN,NaN,NaN,11.784524
KWT,Kuwait,KW,414,ISO 3166-2:KW,Asia,Western Asia,NaN,142.0,145.0,NaN,NaN,NaN,NaN,NaN,11.762863


In [205]:
v = "hd35"

wb_files = os.listdir(wb_path)
for i, f in enumerate(wb_files):
    if f[0] == "~": wb_files.remove(f)
for i, f in enumerate(wb_files): print(f"{i}: {f}")

# population
i = 6
pop_h = pd.read_excel(wb_path + wb_files[i])
pop_h[ix] = pop_h["code"].apply(lambda x: x[0:3])
pop_h = pop_h.set_index(ix)
for n in ["HKG", "MAC", "TWN"]: 
    pop_h.loc[n, "code"] = f"{n}.01"
pop_h_tot = pop_h.copy()
pop_h_tot = pop_h_tot[ (pop_h_tot["code"].apply(lambda x: len(x))==3) ]
pop_h = pop_h[ ~(pop_h["code"].apply(lambda x: len(x))==3) ]
pop_h_tot["1995-07"] = pop_h["1995-07"].groupby(ix).sum()
pop_h["% Population"] = ( pop_h["1995-07"] / pop_h_tot["1995-07"] ).values
pop_h = pop_h.reset_index().set_index("code")

i = 11
pop_f = pd.read_excel(wb_path + wb_files[i])
pop_f[ix] = pop_f["code"].apply(lambda x: x[0:3])
pop_f = pop_f.set_index(ix)
for n in ["HKG", "MAC", "TWN"]: 
    pop_f.loc[n, "code"] = f"{n}.01"
pop_f_tot = pop_f.copy()
pop_f_tot = pop_f_tot[ (pop_f_tot["code"].apply(lambda x: len(x))==3) ]
pop_f = pop_f[ ~(pop_f["code"].apply(lambda x: len(x))==3) ]
pop_f_tot["2040-07"] = pop_f["2040-07"].groupby(ix).sum()
pop_f["% Population"] = ( pop_f["2040-07"] / pop_f_tot["2040-07"] ).values
pop_f = pop_f.reset_index().set_index("code")

pop_f

0: cmip6-x0.25_climatology_pr_climatology_annual_1995-2014_median_historical_ensemble_all_mean.xlsx
1: cmip6-x0.25_climatology_hd35_climatology_annual_2080-2099_median_ssp585_ensemble_all_mean.xlsx
2: cmip6-x0.25_climatology_cdd_climatology_annual_2040-2059_median_ssp245_ensemble_all_mean.xlsx
3: cmip6-x0.25_climatology_rx1day_climatology_annual_1995-2014_median_historical_ensemble_all_mean.xlsx
4: cmip6-x0.25_climatology_pr_climatology_annual_2040-2059_median_ssp245_ensemble_all_mean.xlsx
5: cmip6-x0.25_climatology_tx84rr_climatology_annual_1995-2014_median_historical_ensemble_all_mean.xlsx
6: pop-x1_climatology_popcount_climatology_annual_1995-2014_mean_historical_gpw-v4_rev11_mean.xlsx
7: cmip6-x0.25_climatology_hd35_climatology_annual_1995-2014_median_historical_ensemble_all_mean.xlsx
8: cmip6-x0.25_climatology_rx1day_climatology_annual_2040-2059_median_ssp245_ensemble_all_mean.xlsx
9: cmip6-x0.25_climatology_tx84rr_climatology_annual_2040-2059_median_ssp245_ensemble_all_mean.xlsx


,alpha-3,name,2040-07,% Population
code,,,,
ABW.14465,ABW,Administrative unit not available,2081.473,1.000000
AFG.11,AFG,Badakhshan,928179.500,0.035599
AFG.110,AFG,Ghor,558479.940,0.021420
AFG.111,AFG,Hilmand,829134.000,0.031801
AFG.112,AFG,Hirat,1550612.800,0.059472
...,...,...,...,...
ZWE.2713441,ZWE,Masvingo,1256122.200,0.093380
ZWE.2713443,ZWE,Matabeleland South,710630.200,0.052828
ZWE.2713444,ZWE,Midlands,1290731.900,0.095953


In [206]:
# tx84rr
i = 7
wb_h = pd.read_excel(wb_path + wb_files[i])
wb_h[ix] = wb_h["code"].apply(lambda x: x[0:3])
wb_h = wb_h.set_index(ix)
for n in ["HKG", "MAC"]: 
    wb_h.loc[n, "code"] = f"{n}.01"
wb_h = wb_h[ ~(wb_h["code"].apply(lambda x: len(x))==3) ]
wb_h = wb_h.reset_index().set_index("code")

i = 1
wb_f = pd.read_excel(wb_path + wb_files[i])
wb_f[ix] = wb_f["code"].apply(lambda x: x[0:3])
wb_f = wb_f.set_index(ix)
for n in ["HKG", "MAC"]: 
    wb_f.loc[n, "code"] = f"{n}.01"
wb_f_tot = wb_f.copy()
wb_f_tot = wb_f_tot[ (wb_f_tot["code"].apply(lambda x: len(x))==3) ]
wb_f = wb_f[ ~(wb_f["code"].apply(lambda x: len(x))==3) ]
wb_f = wb_f.reset_index().set_index("code")
wb_f = wb_f[ wb_f["2080-07"] > 7*8 ]
pop_f_i = pop_f.loc[ wb_f.index ]
#wb_f["2080-07"] = wb_f["2080-07"] * pop_f["% Population"]
pop_f_i = pop_f_i.reset_index().set_index(ix)
#wb_f_tot["2080-07"] = wb_f["2080-07"].groupby(ix).sum()
v = "% Population exposed to over 8 weeks of days above 35°C"
pop_f_tot[v] = pop_f_i["% Population"].groupby(ix).sum()
pop_f_tot[v] = pop_f_tot[v].fillna(0)

pop_f_tot

,code,name,2040-07,% Population exposed to over 8 weeks of days above 35°C
alpha-3,,,,
ABW,ABW,Aruba (Neth.),2.081473e+03,0.000000
AFG,AFG,Afghanistan,2.607298e+07,0.565384
AGO,AGO,Angola,1.801091e+07,0.488683
AIA,AIA,Anguilla (U.K.),6.753154e+03,0.000000
ALA,ALA,Finland,4.138513e+04,0.000000
...,...,...,...,...
WSM,WSM,Samoa,3.890474e+04,0.000000
YEM,YEM,Republic of Yemen,1.776707e+07,0.650625
ZAF,ZAF,South Africa,4.203184e+07,0.314014


In [181]:
(pop_f_tot["2040-07"] * pop_f_i["% Population"]).groupby(ix).sum()

alpha-3
ABW    0.000000e+00
AFG    1.474124e+07
AGO    8.801626e+06
AIA    0.000000e+00
ALA    0.000000e+00
           ...     
WSM    0.000000e+00
YEM    1.155970e+07
ZAF    1.319859e+07
ZMB    8.837269e+06
ZWE    1.301776e+07
Length: 244, dtype: float64

In [160]:
i = 1
wb_f = pd.read_excel(wb_path + wb_files[i])
wb_f[ix] = wb_f["code"].apply(lambda x: x[0:3])
wb_f = wb_f.set_index(ix)
wb_f[ wb_f["2080-07"] > 7*8 ]

,code,name,2080-07
alpha-3,,,
AFG,AFG,Afghanistan,93.45
AFG,AFG.111,Hilmand,181.01
AFG,AFG.112,Hirat,110.48
AFG,AFG.113,Jawzjan,145.05
AFG,AFG.114,Kabul,59.05
...,...,...,...
ZWE,ZWE.2713441,Masvingo,126.09
ZWE,ZWE.2713443,Matabeleland South,131.39
ZWE,ZWE.2713444,Midlands,101.56
